In [42]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from scipy.special import rel_entr
import numpy as np

In [43]:
validate_processed_sample = 'data/processed/validate_processed_sample.csv'
validate_processed_target = 'data/processed/validate_processed_target.csv'
validate_processed_diff = 'data/processed/validate_processed_differential_diagnosis.csv'

In [44]:
X_val = pd.read_csv(validate_processed_sample)
y_val = pd.read_csv(validate_processed_target)

dval = xgb.DMatrix(X_val, label=y_val)

del X_val

diff_val = pd.read_csv(validate_processed_diff)
diff_val = diff_val.to_numpy()
y_val = y_val.to_numpy()

In [45]:
bst = xgb.Booster()
model_filename = 'model/xgboost_model_7749.json'
bst.load_model(model_filename)

In [46]:
y_pred_prob = bst.predict(dval)

In [47]:
threshold = 0.01

DDrecall = 0
DDprecision = 0
for i in range(len(y_pred_prob)):
    overlap = 0
    diff_num = 0 
    pred_num = 0
    for pathology in range(49):
        if (y_pred_prob[i][pathology] > threshold):
            pred_num += 1
        if (diff_val[i][pathology] > threshold):
            diff_num += 1

            if (y_pred_prob[i][pathology] > threshold):
                overlap += 1
    DDrecall += overlap/diff_num
    DDprecision += overlap/pred_num

DDrecall /= len(y_pred_prob)
DDprecision /= len(y_pred_prob)
F1 = (2 * DDprecision * DDrecall) / (DDprecision + DDrecall)

In [48]:
print(DDrecall)
print(DDprecision)
print(F1)

0.21422449951684627
0.9979935521865185
0.35273302346517615


In [49]:
print(y_pred_prob[0]>0.01)
print(diff_val[0]>0.01)
# print(np.argmax(y_pred_prob[0]))
# print(np.argmax(diff_val[0]))
print(np.sum(np.logical_and((y_pred_prob[0]>0.01), diff_val[0]>0.01)))

top_indices = np.argsort(y_pred_prob[0])[-8:]
result = np.zeros_like(y_pred_prob[0])
result[top_indices] = y_pred_prob[0][top_indices]
print(np.round(result, 6))

[False False False  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False]
[ True False False  True False False False False False  True False  True
 False False False  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False  True False False  True False  True False
 False]
1
[0.0000e+00 0.0000e+00 0.0000e+00 9.9999e-01 0.0000e+00 1.0000e-06
 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.0000e-06
 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e

In [50]:
from sklearn.metrics import accuracy_score
predicted_classes = np.argmax(diff_val, axis=1)
score = accuracy_score(y_val, predicted_classes)
print(score)

0.7328234476926794
